Improving Model Performance Using MACEst on the Bank Marketing Dataset 
===

This notebook will test weather MACEst can be used in order to improve the ROC-AUC score of a trained model on the bank marketing dataset.

# Training Probabilistic XGBoost on the Bank Marketing Dataset

Because MACEst requires data for training the 'confidence model', we'll keep some data 

## Data Loading

In [ ]:
# todo